In [19]:
import pandas as pd
import re
from datasets import Dataset

# Load Dataset

In [5]:
data_path = '../Data/naruto.csv'
naruto_transcript_df = pd.read_csv(data_path)

In [8]:
#Remove action statements from transcripts
def remove_paranthesis(text):
    result = re.sub(r'\(.*?\)', '', text)
    
    return result

In [9]:
naruto_transcript_df['line'] = naruto_transcript_df['line'].apply(remove_paranthesis)

In [11]:
#Count number of words per line to get rid of lines with too few words
naruto_transcript_df['num_of_words'] = naruto_transcript_df['line'].str.strip().str.split(' ')
naruto_transcript_df['num_of_words'] = naruto_transcript_df['num_of_words'].apply(lambda x: len(x))

In [13]:
#Get all responses from chosen character that have more than 5 words
naruto_transcript_df['naruto_response_flag'] = 0
naruto_transcript_df.loc[(naruto_transcript_df['name'] == 'Naruto')&(naruto_transcript_df['num_of_words'] >5), 'naruto_response_flag'] = 1

In [15]:
#Exclude character's first instance of speech since it isn't a response
idxs_to_take = list(naruto_transcript_df[(naruto_transcript_df['naruto_response_flag'] == 1)&(naruto_transcript_df.index > 0)].index)

In [17]:
system_prompt = """ You are Naruto from the anime "Naruto". Your responses should reflect his personality and speech patterns \n"""
prompts = []
for idx in idxs_to_take:
    prompt = system_prompt
    
    #Get prompts that were said to Naruto from the dataset
    prompt += naruto_transcript_df.iloc[idx - 1]['line']
    prompt += '\n'
    #Get Naruto's response from the prompt in the dataset
    prompt += naruto_transcript_df.iloc[idx]['line']
    prompts.append(prompt)

In [18]:
df = pd.DataFrame({'prompt': prompts})
df.head()

,prompt
0,"You are Naruto from the anime ""Naruto"". Your ..."
1,"You are Naruto from the anime ""Naruto"". Your ..."
2,"You are Naruto from the anime ""Naruto"". Your ..."
3,"You are Naruto from the anime ""Naruto"". Your ..."
4,"You are Naruto from the anime ""Naruto"". Your ..."


In [20]:
dataset = Dataset.from_pandas(df)